# Coastal Flood Risk

Notebook environment to migrate netcdf files to zarr and geojson

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [3]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma
import math
from shapely import wkb

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "FASTTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

<IPython.core.display.Javascript object>

In [4]:
# Project paths & files (manual input)
ds_dir = coclico_data_dir.joinpath("07_flood_risk_jrc")
ds_path1 = ds_dir.joinpath("ExpectedAnnualDamage.nc")
ds_path2 = ds_dir.joinpath("ExpectedAnnualDamageperGDP.nc")
ds_path3 = ds_dir.joinpath("ExpectedAnnualPeopleAffected.nc")
ds_out_file = "Coastal_Flood_risk_Europe"
CF_dir = coclico_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [9]:
# open datasets
ds1 = xr.open_dataset(ds_path1)
ds2 = xr.open_dataset(ds_path2)
ds3 = xr.open_dataset(ds_path3)

# check original dataset
ds1

<xarray.Dataset>
Dimensions:   (row: 24)
Dimensions without coordinates: row
Data variables:
    lat       (row) float64 ...
    lon       (row) float64 ...
    base2000  (row) float64 ...
    sust2050  (row) float64 ...
    sust2100  (row) float64 ...
    frag2050  (row) float64 ...
    frag2100  (row) float64 ...
    ffbd2050  (row) float64 ...
    ffbd2100  (row) float64 ...
Attributes:
    title:        Expected Annual Damage
    description:  Ensemble median Expected Annual Damage (expressed in billio...
    source:       The data comes from: Vousdoukas, Michail; Mentaschi, Lorenz...

<IPython.core.display.Javascript object>

In [ ]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_path1, working_dir=CF_dir)

In [ ]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_path1, working_dir=CF_dir)

In [ ]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_path2, working_dir=CF_dir)

In [ ]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_path2, working_dir=CF_dir)

In [ ]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_path3, working_dir=CF_dir)

In [ ]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_path3, working_dir=CF_dir)

### Add NUTS regions

The nuts regions are not included as attributes in the netCDF files. The NetCDF files only contain lon and lat values at the centroid of NUTS2 regions so we retrieve that information from eurostat ("https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts").

In [6]:
# load nuts regions, filter on countries and replace the geomtry by buffered centroids
nuts_regions = gpd.read_file(
    coclico_data_dir.joinpath("XX_NUTS", "NUTS_RG_20M_2021_4326.geojson")
)
nuts_regions = nuts_regions.to_crs("EPSG:4326")

NUTS0 = nuts_regions[nuts_regions.LEVL_CODE == 0]  # countries
NUTS0["polygons"] = NUTS0.geometry  # rename the geometry
NUTS0["geometry"] = NUTS0.centroid.buffer(0.5)  # set the centroid as geometry

c:\Users\kras\AppData\Local\mambaforge\envs\coclico\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\kras\AppData\Local\Temp\ipykernel_6068\1961265729.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  NUTS0["geometry"] = NUTS0.centroid.buffer(0.5)  # set the centroid as geometry
C:\Users\kras\AppData\Local\Temp\ipykernel_6068\1961265729.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation

<IPython.core.display.Javascript object>

In [10]:
# create dummy file from xarray dataset with lon and lat centroids
ds_d = gpd.GeoDataFrame(geometry=gpd.points_from_xy(ds1.lon, ds1.lat), crs="EPSG:4326",)

# spatial join to keep only nuts regions used ds1
sjoins = gpd.sjoin(ds_d, NUTS0, how="left")

# spatial join alterations (i.e. renaming the columns and reducing the size)
sjoins["instance"] = sjoins.index_right.values
cropped_list = ["instance", "NUTS_ID", "NAME_LATN", "CNTR_CODE", "polygons"]
renamed_list = ["instance", "acronym", "name", "country", "geometry"]
sjoins = sjoins[cropped_list]
sjoins = sjoins.rename(
    {
        "NUTS_ID": renamed_list[1],
        "NAME_LATN": renamed_list[2],
        "CNTR_CODE": renamed_list[3],
        "polygons": renamed_list[4],
    },
    axis="columns",
)

# supplementing the faulty joins
supplement = ["FR", "NO"]
nan_idx = []
for idx, i in enumerate(sjoins.instance):
    if math.isnan(i) == True:
        nan_idx.append(idx)

for i, j in zip(nan_idx, supplement):
    supp = NUTS0.loc[NUTS0["NUTS_ID"] == j]
    for kid, (k, l) in enumerate(zip(renamed_list, cropped_list)):
        if kid == 0:
            sjoins.at[i, k] = supp.index.values[0]
        else:
            sjoins.at[i, k] = supp[l].values[0]

sjoins = gpd.GeoDataFrame(sjoins, crs="EPSG:4326")
sjoins.head()

,instance,acronym,name,country,geometry
0,5.0,BE,Belgique/België,BE,"MULTIPOLYGON (((4.75993 51.50246, 4.86400 51.4..."
1,6.0,BG,Bulgaria,BG,"MULTIPOLYGON (((22.99717 43.80787, 23.40948 43..."
2,8.0,CY,Kýpros,CY,"MULTIPOLYGON (((32.27382 35.06649, 32.28884 35..."
3,10.0,DE,Deutschland,DE,"MULTIPOLYGON (((9.11310 54.87360, 9.29405 54.8..."
4,11.0,DK,Danmark,DK,"MULTIPOLYGON (((15.15353 55.07771, 15.08181 54..."


<IPython.core.display.Javascript object>

In [11]:
sjoins

,instance,acronym,name,country,geometry
0,5.0,BE,Belgique/België,BE,"MULTIPOLYGON (((4.75993 51.50246, 4.86400 51.4..."
1,6.0,BG,Bulgaria,BG,"MULTIPOLYGON (((22.99717 43.80787, 23.40948 43..."
2,8.0,CY,Kýpros,CY,"MULTIPOLYGON (((32.27382 35.06649, 32.28884 35..."
3,10.0,DE,Deutschland,DE,"MULTIPOLYGON (((9.11310 54.87360, 9.29405 54.8..."
4,11.0,DK,Danmark,DK,"MULTIPOLYGON (((15.15353 55.07771, 15.08181 54..."
5,12.0,EE,Eesti,EE,"MULTIPOLYGON (((25.83016 59.56406, 25.89485 59..."
6,21.0,EL,Elláda,EL,"MULTIPOLYGON (((27.88675 35.95839, 27.74545 35..."
7,23.0,ES,España,ES,"MULTIPOLYGON (((4.25776 39.96463, 4.30652 39.8..."
8,24.0,FI,Suomi/Finland,FI,"MULTIPOLYGON (((28.92968 69.05190, 28.62465 68..."
9,0.0,FR,France,FR,"MULTIPOLYGON (((55.21471 -21.03851, 55.27392 -..."


<IPython.core.display.Javascript object>

In [12]:
# add geometries

# extract geometries of nut2 regions in well-known binary format
geoms = sjoins["geometry"].apply(lambda x: wkb.dumps(x))

# rename dims and add new data to dataset
ds1 = ds1.assign_coords({"geometry": ("row", geoms)})
ds2 = ds2.assign_coords({"geometry": ("row", geoms)})
ds3 = ds3.assign_coords({"geometry": ("row", geoms)})

<IPython.core.display.Javascript object>

In [13]:
add_geom_attrs = {
    "geometry": {
        "long_name": "NUTS2 regions (polygons) in well-known binary format (wkb).",
        "geometry_type": "polygon",
        "units": "degree",
        "comment": "These NUTS2 regions (2021 version) are available at Eurostat.",
        "crs_wkt": f"{sjoins.crs.to_epsg()}",
    },
}

for k, v in add_geom_attrs.items():
    ds1[k].attrs = add_geom_attrs[k]
    ds2[k].attrs = add_geom_attrs[k]
    ds3[k].attrs = add_geom_attrs[k]

<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [ ]:
# NetCDF attribute, variable and dimension alterations

# set lon/lat coordinates for each of the datasets
ds1, ds2, ds3 = [ds.set_coords(["lon", "lat"]) for ds in [ds1, ds2, ds3]]

# add global attributes
ds1.attrs["Conventions"] = "CF-1.8"
ds2.attrs["Conventions"] = "CF-1.8"
ds3.attrs["Conventions"] = "CF-1.8"
ds1.attrs["crs"] = 4326
ds2.attrs["crs"] = 4326
ds3.attrs["crs"] = 4326

# rename dimensions
ds1 = ds1.rename_dims({"row": "stations"})
ds2 = ds2.rename_dims({"row": "stations"})
ds3 = ds3.rename_dims({"row": "stations"})

# alter variable attributes
ds_list = [ds1, ds2, ds3]
for i in range(3):

    ds_list[i].lat.attrs["long_name"] = ds_list[i].lat.attrs["standard_name"]
    ds_list[i].lat.attrs["standard_name"] = "latitude"
    ds_list[i].lat.attrs["units"] = "degrees_north"

    ds_list[i].lon.attrs["long_name"] = ds_list[i].lon.attrs["standard_name"]
    ds_list[i].lon.attrs["standard_name"] = "latitude"
    ds_list[i].lon.attrs["units"] = "degrees_north"

    ds_list[i].base2000.attrs["long_name"] = ds_list[i].base2000.attrs["standard_name"]
    del ds_list[i].base2000.attrs["standard_name"]
    ds_list[i].sust2050.attrs["long_name"] = ds_list[i].sust2050.attrs["standard_name"]
    del ds_list[i].sust2050.attrs["standard_name"]
    ds_list[i].sust2100.attrs["long_name"] = ds_list[i].sust2100.attrs["standard_name"]
    del ds_list[i].sust2100.attrs["standard_name"]
    ds_list[i].frag2050.attrs["long_name"] = ds_list[i].frag2050.attrs["standard_name"]
    del ds_list[i].frag2050.attrs["standard_name"]
    ds_list[i].frag2100.attrs["long_name"] = ds_list[i].frag2100.attrs["standard_name"]
    del ds_list[i].frag2100.attrs["standard_name"]
    ds_list[i].ffbd2050.attrs["long_name"] = ds_list[i].ffbd2050.attrs["standard_name"]
    del ds_list[i].ffbd2050.attrs["standard_name"]
    ds_list[i].ffbd2100.attrs["long_name"] = ds_list[i].ffbd2100.attrs["standard_name"]
    del ds_list[i].ffbd2100.attrs["standard_name"]

    if i == 0 or i == 2:  # annual damage
        ds_list[i].base2000.attrs["units"] = "1e12"
        ds_list[i].sust2050.attrs["units"], ds_list[i].sust2100.attrs["units"] = (
            "1e12",
            "1e12",
        )
        ds_list[i].frag2050.attrs["units"], ds_list[i].frag2100.attrs["units"] = (
            "1e12",
            "1e12",
        )
        ds_list[i].ffbd2050.attrs["units"], ds_list[i].ffbd2100.attrs["units"] = (
            "1e12",
            "1e12",
        )
    if i == 1:  # annual damage per GDP
        ds_list[i].base2000.attrs["units"] = "1"
        ds_list[i].sust2050.attrs["units"], ds_list[i].sust2100.attrs["units"] = (
            "1",
            "1",
        )
        ds_list[i].frag2050.attrs["units"], ds_list[i].frag2100.attrs["units"] = (
            "1",
            "1",
        )
        ds_list[i].ffbd2050.attrs["units"], ds_list[i].ffbd2100.attrs["units"] = (
            "1",
            "1",
        )
    if i == 2:  # annual people affected
        ds_list[i].base2000.attrs["units"] = "1e3"
        ds_list[i].sust2050.attrs["units"], ds_list[i].sust2100.attrs["units"] = (
            "1e3",
            "1e3",
        )
        ds_list[i].frag2050.attrs["units"], ds_list[i].frag2100.attrs["units"] = (
            "1e3",
            "1e3",
        )
        ds_list[i].ffbd2050.attrs["units"], ds_list[i].ffbd2100.attrs["units"] = (
            "1e3",
            "1e3",
        )

In [ ]:
# NetCDF variable and dimension alterations

# concatenate the datasets for time and scenarios
ds_list_new = []
var_list = [
    "expected annual damage",
    "expected annual damage per GDP",
    "expected annual people affected",
]
var_list_abb = ["ead", "ead_GDP", "eapa"]
for i in range(3):
    das = xr.concat(
        [ds_list[i]["base2000"], ds_list[i]["sust2050"], ds_list[i]["sust2100"]],
        dim="time",
    )
    das = das.assign_coords({"time": ("time", np.array([2000, 2050, 2100]))})
    das.time.attrs["long_name"] = "time"
    das.time.attrs["units"] = "yr"
    daf = xr.concat(
        [ds_list[i]["base2000"], ds_list[i]["frag2050"], ds_list[i]["frag2100"]],
        dim="time",
    )
    daf = daf.assign_coords({"time": ("time", np.array([2000, 2050, 2100]))})
    daf.time.attrs["long_name"] = "time"
    daf.time.attrs["units"] = "yr"
    dag = xr.concat(
        [ds_list[i]["base2000"], ds_list[i]["ffbd2050"], ds_list[i]["ffbd2100"]],
        dim="time",
    )
    dag = dag.assign_coords({"time": ("time", np.array([2000, 2050, 2100]))})
    dag.time.attrs["long_name"] = "time"
    dag.time.attrs["units"] = "yr"

    dsnew = xr.concat([das, daf, dag], dim="nscenarios")
    dsnew = dsnew.assign_coords(
        {
            "scenarios": (
                "nscenarios",
                np.array(["RCP4.5-SSP1", "RCP8.5-SSP3", "RCP8.5-SSP5",], dtype="S",),
            )
        }
    )
    dsnew.scenarios.attrs["long_name"] = "climate scenarios"
    dsnew = dsnew.to_dataset(name=var_list_abb[i])
    dsnew[var_list_abb[i]].attrs["long_name"] = var_list[i]  # variable attributes
    dsnew.attrs = ds1.attrs  # copy global attributes
    ds_list_new.append(dsnew)

# merge into one dataset
ds = xr.merge(ds_list_new)

In [ ]:
# re-order shape of the data variables
ds1_new = ds_list_new[0].transpose("nscenarios", "stations", "time")
ds2_new = ds_list_new[1].transpose("nscenarios", "stations", "time")
ds3_new = ds_list_new[2].transpose("nscenarios", "stations", "time")
ds = ds.transpose("nscenarios", "stations", "time")

In [ ]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

ds
# ds["scenarios"]

In [ ]:
# save new .nc files
ds1_new.to_netcdf(path=str(ds_path1).replace(".nc", "_CF.nc"))
ds2_new.to_netcdf(path=str(ds_path2).replace(".nc", "_CF.nc"))
ds3_new.to_netcdf(path=str(ds_path3).replace(".nc", "_CF.nc"))
ds.to_netcdf(path=ds_dir.joinpath(ds_out_file + "_CF.nc"))

### Check CF compliancy altered NetCDF files

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(ds_path1).replace(".nc", "_CF.nc"), working_dir=CF_dir)

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(ds_path1).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(ds_path2).replace(".nc", "_CF.nc"), working_dir=CF_dir)

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(ds_path2).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(ds_path3).replace(".nc", "_CF.nc"), working_dir=CF_dir)

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(ds_path3).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir)

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir,
)

### write data to Zarr files

In [ ]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_file), mode="w")